In [1]:
import re
import gc
import os
import pickle
import random
from preprocess import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.model_selection import KFold,StratifiedKFold, train_test_split
import tensorflow as tf
from sklearn.utils import shuffle
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix, roc_curve
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report,f1_score
from tensorflow.keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D, GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional, LSTM

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth',-1)

C:\Users\rrahimi\AppData\Local\Temp\ipykernel_1756\1494163217.py:26: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth',-1)


In [2]:
df_ = pd.read_csv(r'D:\ترم دوم\تحلیل احساس\persian-sentiment-analysis-using-fastText-word-embedding-and-pseudo-labeling-main\persian-sentiment-analysis-using-fastText-word-embedding-and-pseudo-labeling-main\sample_dataset_10000.csv', encoding='utf-8')

In [3]:
df_.head()

,BadComment,Comment,Date,GoodComment,ItemURL,NegetiveFeedback,Person,PositiveFeedback,SubCatName,SubName
0,۰,<p>من کاملا با این محصول آشنا بودم و از خریدش مطمین بودم</p>,\r\r\n در تاریخ ۲ مهر ۱۳۹۹\r\r\n,۰,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\r\n سمانه خلج مهرآبادي\r\r\n,1,ساعت هوشمند,کالای دیجیتال
1,۲,<p>به درد خاصی نمیخوره صرفا یکی دو روز باهاش سرگرمین بعد به عنوان یه ساعت معمولی ازش استفاده میکنین</p>,\r\r\n در تاریخ ۱ مهر ۱۳۹۹\r\r\n,۴,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\r\n محجوبه زارعی\r\r\n,0,ساعت هوشمند,کالای دیجیتال
2,۰,<p>در یک کلام به نسبت هزینه ای که میکنید کالمل ترین ساعت بازار رو دریافت می‌کنید<br/>\r\r\r\nدسترسی کامل به نوتیفیکشن ها و توانایی به پاسخ دادن به پیام ها از طریق خود ساعت<br/>\r\r\r\nهمچنین برنامه کامل ورزشی تمام نیاز های شما رو برآورده میکنه</p>,\r\r\n در تاریخ ۳۱ شهریور ۱۳۹۹\r\r\n,۰,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\r\n محمد رضا میرشکار سیاهکل\r\r\n,0,ساعت هوشمند,کالای دیجیتال
3,۰,<p>ساعت بسیار عالی و کاربردی هست،میخواستم برای دوستم هم بگیرم ولی به دلیل تفاوت قیمت زیاد از دیجی کالا نمیخرم،</p>,\r\r\n در تاریخ ۲۹ شهریور ۱۳۹۹\r\r\n,۰,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\r\n کاربر دیجی‌کالا\r\r\n,0,ساعت هوشمند,کالای دیجیتال
4,۰,<p>دقیقا مطابق آنچه انتظار می رفت بود .<br/>\r\r\r\nخوب و کاربردی و مطابق استایل روز .</p>,\r\r\n در تاریخ ۲۹ شهریور ۱۳۹۹\r\r\n,۱,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\r\n محمد میرفخرایی\r\r\n,1,ساعت هوشمند,کالای دیجیتال


In [4]:
df_['NegetiveFeedback'].value_counts()

0    8902
1    1098
Name: NegetiveFeedback, dtype: int64

In [5]:
df_['len'] = df_['Comment'].astype(str).apply(lambda x: len(x.split()))
df_.head()


,BadComment,Comment,Date,GoodComment,ItemURL,NegetiveFeedback,Person,PositiveFeedback,SubCatName,SubName,len
0,۰,<p>من کاملا با این محصول آشنا بودم و از خریدش مطمین بودم</p>,\r\r\n در تاریخ ۲ مهر ۱۳۹۹\r\r\n,۰,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\r\n سمانه خلج مهرآبادي\r\r\n,1,ساعت هوشمند,کالای دیجیتال,12
1,۲,<p>به درد خاصی نمیخوره صرفا یکی دو روز باهاش سرگرمین بعد به عنوان یه ساعت معمولی ازش استفاده میکنین</p>,\r\r\n در تاریخ ۱ مهر ۱۳۹۹\r\r\n,۴,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\r\n محجوبه زارعی\r\r\n,0,ساعت هوشمند,کالای دیجیتال,19
2,۰,<p>در یک کلام به نسبت هزینه ای که میکنید کالمل ترین ساعت بازار رو دریافت می‌کنید<br/>\r\r\r\nدسترسی کامل به نوتیفیکشن ها و توانایی به پاسخ دادن به پیام ها از طریق خود ساعت<br/>\r\r\r\nهمچنین برنامه کامل ورزشی تمام نیاز های شما رو برآورده میکنه</p>,\r\r\n در تاریخ ۳۱ شهریور ۱۳۹۹\r\r\n,۰,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\r\n محمد رضا میرشکار سیاهکل\r\r\n,0,ساعت هوشمند,کالای دیجیتال,44
3,۰,<p>ساعت بسیار عالی و کاربردی هست،میخواستم برای دوستم هم بگیرم ولی به دلیل تفاوت قیمت زیاد از دیجی کالا نمیخرم،</p>,\r\r\n در تاریخ ۲۹ شهریور ۱۳۹۹\r\r\n,۰,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\r\n کاربر دیجی‌کالا\r\r\n,0,ساعت هوشمند,کالای دیجیتال,20
4,۰,<p>دقیقا مطابق آنچه انتظار می رفت بود .<br/>\r\r\r\nخوب و کاربردی و مطابق استایل روز .</p>,\r\r\n در تاریخ ۲۹ شهریور ۱۳۹۹\r\r\n,۱,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\r\n محمد میرفخرایی\r\r\n,1,ساعت هوشمند,کالای دیجیتال,16


In [6]:
print(len(df_))
df = df_[(df_['PositiveFeedback']==1)|(df_['NegetiveFeedback']==1)]
print(len(df))

10000
7295


In [7]:
df = df.sample(frac=1.0)

In [8]:
df['Label'] = 0
df.loc[df['NegetiveFeedback']==1,'Label'] = 1
df.head()

,BadComment,Comment,Date,GoodComment,ItemURL,NegetiveFeedback,Person,PositiveFeedback,SubCatName,SubName,len,Label
5875,۰,<p>عالی واقعاخیلی خوبه حتما پیشنهاد میکنم</p>,\r\r\n در تاریخ ۱۲ اردیبهشت ۱۳۹۹\r\r\n,۱,/product/dkp-1108677/ساعت-هوشمند-میدسان-مدل-a1,0,\r\r\n رسول اکبری\r\r\n,1,ساعت هوشمند,کالای دیجیتال,6,0
8672,۰,<p>دستبند فوق العاده خوبیه<br/>\r\r\r\nشک نکنید و بخرید<br/>\r\r\r\nعالیییی...<br/>\r\r\n<br/>\r\r\r\nراه حل برای مشکل واچ فیس<br/>\r\r\r\n۱- اگه گوشی هواوی دارین فقط Huawei health رو حذف کنید و یا حذف دیتا برنامه رو بزنید. (بهتره کشور برنامه های مثل care و app گالری رو بزارید روی United Kingdom و همینطور یه هواوی آیدی هم با کشور United Kingdom داشته باشد، البته فکر کنم این آیدی هم نباشه مشکلی ایجاد نشه، ولی اگه مشکلی داشت دوباره امتحان کنید با آیدی) بعد huawei health رو از گوگل پلی دانلود کنید وقتی نصب شد سریع اینترنت رو خاموش کنید و حالت هواپیما رو فعال کنید. بعد برید توی huawei health. یه صفحه میاد که میگه کشور رو انتخاب کنید، شما بزارید روی United Kingdom یا انگلیس،، بعد برنامه باز میشه و بعد تنظیمات وصل شدن به دستبند رو انجام بدین. بعد که دستبند وصل شد گزینه واچ فیس توی تنظیمات دستبند اضافه شده و میتونید واچ فیس اضافه کنید. یا عوض کنید. <br/>\r\r\r\n۲- اونایی که گوشی هواوی ندارین اول huawei mobile servises و بعد health رو از گوگل پلی نصب کنید و بعد مراحل بالا رو انجام بدین.</p>,\r\r\n در تاریخ ۲۲ فروردین ۱۳۹۹\r\r\n,۰,/product/dkp-2022788/مچ-بند-هوشمند-آنر-مدل-band-5-global,0,\r\r\n رضا زرین\r\r\n,1,ساعت هوشمند,کالای دیجیتال,186,0
5428,۰,<p>نسبت به قیمت بد نیست ولی متاسفانه نرم افزارش چینیه و هیچی نمیشه فهمید</p>,\r\r\n در تاریخ ۴ شهریور ۱۳۹۹\r\r\n,۰,/product/dkp-1108677/ساعت-هوشمند-میدسان-مدل-a1,0,\r\r\n محسن سلیمی\r\r\n,1,ساعت هوشمند,کالای دیجیتال,14,0
7552,۰,<p>سبک، و دقیق</p>,\r\r\n در تاریخ ۱۸ مرداد ۱۳۹۹\r\r\n,۰,/product/dkp-2022788/مچ-بند-هوشمند-آنر-مدل-band-5-global,0,\r\r\n عبدالوهاب یوسفی\r\r\n,1,ساعت هوشمند,کالای دیجیتال,3,0
2384,۱,<p>خوب است ولی خدمات پس از فروش ندارد .</p>,\r\r\n در تاریخ ۲۸ مرداد ۱۳۹۹\r\r\n,۱,/product/dkp-2264825/ساعت-هوشمند-مودیو-مدل-mw01,0,\r\r\n رسول همت زاده\r\r\n,1,ساعت هوشمند,کالای دیجیتال,9,0


In [9]:
df['Label'].value_counts()

0    6197
1    1098
Name: Label, dtype: int64

In [10]:
#prep = preprocess.clean()
df['Comment'] = df['Comment'].apply(lambda x: clean(x))
  


In [11]:
df['Comment'][1:10]

8672    دستبند فوق العاده خوبیه  شک نکنید و بخرید  عالیییی.   راه حل برای مشکل واچ فیس  اگه گوشی هواوی دارین فقط Huawei health رو حذف کنید و یا حذف دیتا برنامه رو بزنید . بهتره کشور برنامه های مثل care و app گالری رو بزارید روی United Kingdom و همینطور یه هواوی آیدی هم با کشور United Kingdom داشته باشد البته فکر کنم این آیدی هم نباشه مشکلی ایجاد نشه ولی اگه مشکلی داشت دوباره امتحان کنید با آیدی بعد huawei health رو از گوگل پلی دانلود کنید وقتی نصب شد سریع اینترنت رو خاموش کنید و حالت هواپیما رو فعال کنید . بعد برید توی huawei health. یه صفحه میاد که میگه کشور رو انتخاب کنید شما بزارید روی United Kingdom یا انگلیس بعد برنامه باز میشه و بعد تنظیمات وصل شدن به دستبند رو انجام بدین . بعد که دستبند وصل شد گزینه واچ فیس توی تنظیمات دستبند اضافه شده و میتونید واچ فیس اضافه کنید . یا عوض کنید .  اونایی که گوشی هواوی ندارین اول huawei mobile servises و بعد health رو از گوگل پلی نصب کنید و بعد مراحل بالا رو انجام بدین .
5428    نسبت به قیمت بد نیست ولی متاسفانه نرم افزارش چینیه و هیچی نمیشه فهمی

In [12]:
X = df['Comment'].values


Y = df['Label'].values


In [13]:
embed_num_dims = 100
max_seq_len = 400
EMB_SIZE = 100
MAX_LEN = 1000
seed = 2020
random.seed(seed)
skf = StratifiedKFold(n_splits=5)
batch_size = 2048
epoches = 7

In [48]:
tokenizer = Tokenizer(num_words=2000000,filters='!"#$%&()*+,-./;<=>?@][\\]^{|}~\t\n')
tokenizer.fit_on_texts(X)
index_of_words = tokenizer.word_index

In [53]:
index_of_words.items()

dict_items([('و', 1), ('به', 2), ('که', 3), ('از', 4), ('ساعت', 5), ('خیلی', 6), ('من', 7), ('این', 8), ('با', 9), ('رو', 10), ('هم', 11), ('در', 12), ('داره', 13), ('برای', 14), ('ولی', 15), ('خوبه', 16), ('بود', 17), ('می', 18), ('بند', 19), ('خوب', 20), ('عالی', 21), ('نسبت', 22), ('قیمت', 23), ('هست', 24), ('میشه', 25), ('عالیه', 26), ('های', 27), ('فقط', 28), ('کیفیت', 29), ('خریدم', 30), ('قیمتش', 31), ('گوشی', 32), ('یه', 33), ('استفاده', 34), ('واقعا', 35), ('راضی', 36), ('کنید', 37), ('شارژ', 38), ('میکنم', 39), ('کار', 40), ('خوبی', 41), ('تا', 42), ('کردم', 43), ('یک', 44), ('نیست', 45), ('مچ', 46), ('بسیار', 47), ('تو', 48), ('کالا', 49), ('است', 50), ('خرید', 51), ('فارسی', 52), ('ها', 53), ('نظر', 54), ('همه', 55), ('روز', 56), ('کل', 57), ('میکنه', 58), ('پیشنهاد', 59), ('صفحه', 60), ('دیجی', 61), ('نداره', 62), ('را', 63), ('بعد', 64), ('خوبیه', 65), ('باید', 66), ('ازش', 67), ('باشه', 68), ('اگه', 69), ('امکانات', 70), ('راضیم', 71), ('اما', 72), ('اصلا', 73), ('گرفتم'

In [45]:
def get_coefs(word,*arr):
    return word,np.asarray(arr,dtype='float32')
def build_embed_matrix(embed_path,word_index):
    EMB_SIZE =100
    embed_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embed_path, encoding='utf-8'))
    embed_matrix = np.zeros((len(word_index)+1,EMB_SIZE))
    
    for word , i in word_index.items():
        embed_vector = embed_index.get(word)
        if embed_vector is not None:
            embed_matrix[i]=embed_vector
    del embed_index
    gc.collect()
    return embed_matrix

In [46]:
embed_path = 'D:\ترم دوم\تحلیل احساس\persian-sentiment-analysis-using-fastText-word-embedding-and-pseudo-labeling-main\persian-sentiment-analysis-using-fastText-word-embedding-and-pseudo-labeling-main\DigiKalaEmbeddingVevtors\DigiKalaEmbeddingVevtors.vec'
embed_matrix_fasts = build_embed_matrix(embed_path,index_of_words)

In [56]:
def Evaluation(Ytest, pred_class, pred_proba, fold):
    
    auc = roc_auc_score(Ytest,pred_proba)
    fpr, tpr, threshold = roc_curve(Ytest, pred_proba)
    
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label='AUC = %0.2f' % auc)
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.savefig('pseudolabeled_cleaned_auc'+str(fold)+'.png')
    plt.show()
    print('AUC:', auc)
    print('f1_score:', f1_score(Ytest,pred_class))
    print('Confusion Matrix:\n', confusion_matrix(Ytest,pred_class))

In [57]:
model = Sequential()
model.add(Embedding(len(index_of_words)+1, 100, input_length=max_seq_len, weights=[embed_matrix_fasts]))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 100)          931800    
                                                                 
 dropout (Dropout)           (None, 400, 100)          0         
                                                                 
 bidirectional (Bidirectiona  (None, 64)               34048     
 l)                                                              
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                        

In [58]:
model = Sequential()
model.add(Embedding(len(index_of_words)+1, 100, input_length=max_seq_len, weights=[embed_matrix_fasts]))
model.add(Dropout(0.5))
model.add(Conv1D(128,kernel_size=3,padding='same',activation='relu',strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 400, 100)          931800    
                                                                 
 dropout_4 (Dropout)         (None, 400, 100)          0         
                                                                 
 conv1d (Conv1D)             (None, 400, 128)          38528     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                      

# Bidirectional LSTM


In [59]:
def create_model():
    model = Sequential()
    model.add(Embedding(len(index_of_words)+1, 100, input_length=max_seq_len, weights=[embed_matrix_fasts]))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(32)))
    model.add(Dropout(0.5))
    model.add(Dense(64,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(16,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [ ]:
for fold, (train_index, test_index) in enumerate(skf.split(X,Y)):
    
    print('#Fold:', fold+1)
    
    Xtrain, Xtest = X[train_index], X[test_index]
    Ytrain, Ytest = Y[train_index], Y[test_index]
    
    print('Oversampling ...')
    Xtrain = np.append(Xtrain, Xtrain[Ytrain==1])
    Ytrain = np.append(Ytrain, Ytrain[Ytrain==1])
    
    print('Pad Sequencing ...')
    sequence_train = tokenizer.texts_to_sequences(Xtrain)
    sequence_test = tokenizer.texts_to_sequences(Xtest)
    padata_train = pad_sequences(sequence_train,maxlen=max_seq_len, padding='post')
    padata_test = pad_sequences(sequence_test,maxlen=max_seq_len, padding='post')
    
    print('Modeling ...')
    model = create_model()
    history= model.fit(padata_train, Ytrain, validation_data=(padata_test,Ytest),epochs=epoches, batch_size=batch_size,
                       verbose=1, shuffle=True)
    
    print('Evaluating ...')
   # pred_class=predict_classes(padata_test)
    pred_class = (model.predict(padata_test) > 0.5).astype("int32")
    pred_proba = model.predict(padata_test)
    Evaluation(Ytest, pred_class, pred_proba, fold)

In [60]:
for fold, (train_index, test_index) in enumerate(skf.split(X,Y)):
    
    print('#Fold:', fold+1)
    
    Xtrain, Xtest = X[train_index], X[test_index]
    Ytrain, Ytest = Y[train_index], Y[test_index]
    
    print('Oversampling ...')
    Xtrain = np.append(Xtrain, Xtrain[Ytrain==1])
    Ytrain = np.append(Ytrain, Ytrain[Ytrain==1])
    
    print('Pad Sequencing ...')
    sequence_train = tokenizer.texts_to_sequences(Xtrain)
    sequence_test = tokenizer.texts_to_sequences(Xtest)
    padata_train = pad_sequences(sequence_train,maxlen=max_seq_len, padding='post')
    padata_test = pad_sequences(sequence_test,maxlen=max_seq_len, padding='post')

#Fold: 1
Oversampling ...
Pad Sequencing ...
#Fold: 2
Oversampling ...
Pad Sequencing ...
#Fold: 3
Oversampling ...
Pad Sequencing ...
#Fold: 4
Oversampling ...
Pad Sequencing ...
#Fold: 5
Oversampling ...
Pad Sequencing ...


In [67]:
padata_train

array([[  21, 4526,   16, ...,    0,    0,    0],
       [ 322,  134,  172, ...,    0,    0,    0],
       [  22,    2,   23, ...,    0,    0,    0],
       ...,
       [   9,  354,   31, ...,    0,    0,    0],
       [ 178,   29,    0, ...,    0,    0,    0],
       [  69, 1034,  143, ...,    0,    0,    0]])